In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv openAI langchain_core langchain langchain_openai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 환경변수 설정

In [3]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [4]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [5]:
# 예제 1-18 <`get_format_instructions` 메서드>

# 라이브러리 불러오기
from langchain_core.output_parsers import JsonOutputParser
# Json 출력 파서 불러오기
parser = JsonOutputParser()
instructions = parser.get_format_instructions()
print(instructions)  # JSON 형식의 지침을 출력


Return a JSON object.


In [6]:
# 예제 1-19 <`parse` 메서드>

ai_response = '{"이름": "김철수", "나이": 30}'
parsed_response = parser.parse(ai_response)
print(parsed_response)


{'이름': '김철수', '나이': 30}


In [8]:
# 예제 1-20 <`parse_with_prompt` 메서드>

# 라이브러리 불러오기
from langchain.output_parsers import RetryWithErrorOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
# 파서 설정
parser = RetryWithErrorOutputParser.from_llm(parser=JsonOutputParser(), llm=ChatOpenAI())

question = "가장 큰 대륙은?"
ai_response = "아시아입니다."  # JSON 형식이 아닌 잘못된 응답
try:
    result = parser.parse_with_prompt(ai_response, question)
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")
    # 여기서 AI에게 다시 질문할 수 있습니다


오류 발생: 'str' object has no attribute 'to_string'


In [9]:
# 예제 1-21 <Pydantic 출력 파서 예시>

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator


# OpenAI 모델 설정
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")
    # Pydantic을 사용한 사용자 정의 검증 로직
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup", "")
        if not setup.endswith("?"):
            raise ValueError("잘못된 질문 형식입니다! 질문은 '?'로 끝나야 합니다.")
        return values

# 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n질문: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# 언어 모델을 사용해 데이터 구조를 채우도록 프롬프트와 모델 설정
chain = prompt | model | parser


# 체인 실행 및 결과 출력
try:
    result = chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있는 질문하여라."})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")


setup='부동산 투자를 고려하고 있습니다. 어떤 금융 상품을 선택하는 것이 좋을까요?' advice='부동산 투자를 고려할 때는 주택 담보 대출이나 부동산 투자 전용 대출을 고려해보세요. 또한 부동산 투자의 리스크를 충분히 고려하고 적절한 금융 상품을 선택하는 것이 중요합니다.'


In [11]:
# 예제 1-22 <SimpleJsonOutputParser 출력 파서 예시>

from langchain.output_parsers.json import SimpleJsonOutputParser
# JSON 포맷의 응답을 생성하는 프롬프트 템플릿 설정
json_prompt = PromptTemplate.from_template(
    "다음 질문에 대한 답변이 포함된 JSON 객체를 반환하십시오: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

# 스트리밍 예시: 질문에 대한 답변이 부분적으로 구문 분석됨
list(json_chain.stream({"question": "비트코인에 대한 짧은 한문장 설명."}))


[{},
 {'answer': ''},
 {'answer': '비'},
 {'answer': '비트'},
 {'answer': '비트코'},
 {'answer': '비트코인'},
 {'answer': '비트코인은'},
 {'answer': '비트코인은 분'},
 {'answer': '비트코인은 분산'},
 {'answer': '비트코인은 분산된'},
 {'answer': '비트코인은 분산된 디'},
 {'answer': '비트코인은 분산된 디지'},
 {'answer': '비트코인은 분산된 디지털'},
 {'answer': '비트코인은 분산된 디지털 화'},
 {'answer': '비트코인은 분산된 디지털 화폐'},
 {'answer': '비트코인은 분산된 디지털 화폐로'},
 {'answer': '비트코인은 분산된 디지털 화폐로,'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 사용'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 사용자'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 사용자들'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 사용자들 간'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 사용자들 간의'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 사용자들 간의 직'},
 {'answer': '비트코인은 분산된 디지털 화폐로, 중앙 기관 

In [13]:
# 예제 1-23 <JsonOutputParser출력 파서 예시>

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
# 오픈AI 모델 설정
model = ChatOpenAI(temperature=0)


# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")

# JSON 출력 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = JsonOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 구성: 프롬프트 -> 모델 -> 파서
chain = prompt | model | parser
# 체인 실행
chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있는 질문하여라."})


{'setup': '저는 부동산 투자에 대해 전혀 경험이 없습니다. 어떻게 시작해야 할까요?',
 'advice': '부동산 투자를 시작하려면 먼저 자신의 재정 상황을 분석하고 목표를 설정해야 합니다. 또한 지역별 부동산 시장 조사와 전문가의 조언을 듣는 것이 도움이 될 수 있습니다.'}